In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split , KFold
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

from collections import Counter

Importing necessary packages 
Classifier I chose is KNN also known as K Nearest Neighbour
Also interestingly enough I actually worked with this data before during my studies at the university.
Also this dataset is mentioned in statistical programming and in the book statistical learning with R
Incredibly logistic regression works incredible well too
iris is also default dataset if we work on rstudio
Also iris flower data is a simple example of amarhan supervised learning bodlogo

In [3]:
# We aar going to import our data
iris = datasets.load_iris()
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                      columns= iris['feature_names'] + ['target'])
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [4]:
# Talibar oruulsan ni deer baih
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [5]:
# Datagaa partition hiih
x= iris_df.iloc[:, :-1]
y= iris_df.iloc[:, -1]

In [6]:
# KNN uuruu engiin supervised learning algorithm uchir 
# Our supervised learning model will require as to split our data into training and testing
# Our testing case will 20percent rather than the convential 30percent
x_train, x_test, y_train, y_test= train_test_split(x, y,
                                                   test_size= 0.2,
                                                   shuffle= True,
                                                   random_state= 0)
x_train= np.asarray(x_train)
y_train= np.asarray(y_train)

x_test= np.asarray(x_test)
y_test= np.asarray(y_test)

In [7]:
scaler= Normalizer().fit(x_train) # the scaler is fitted to the training set
normalized_x_train= scaler.transform(x_train) # the scaler is applied to the training set
normalized_x_test= scaler.transform(x_test) # the scaler is applied to the test set

In [8]:
# Euclidean zaig todorhoiloj baigaa ba norm L2 -oor songoj awaj baigaa 
def distance_ecu(x_train, x_test_point):

  distances= []  
  for row in range(len(x_train)):
      current_train_point= x_train[row] 
      current_distance= 0 

      for col in range(len(current_train_point)): 
          
          current_distance += (current_train_point[col] - x_test_point[col]) **2
        
      current_distance= np.sqrt(current_distance)

      distances.append(current_distance) 

  distances= pd.DataFrame(data=distances,columns=['dist'])
  return distances


In [9]:
def nearest_neighbors(distance_point, K):
 
    df_nearest= distance_point.sort_values(by=['dist'], axis=0)
    df_nearest= df_nearest[:K]
    return df_nearest

In [10]:
def voting(df_nearest, y_train):
    counter_vote= Counter(y_train[df_nearest.index])

    y_pred= counter_vote.most_common()[0][0] 

    return y_pred

In [11]:
#Full algorithmaa gargaj baigaa 
def KNN(x_train, y_train, x_test, K):
    y_pred=[]

    for x_test_point in x_test:
      distance_point  = distance_ecu(x_train, x_test_point)  
      df_nearest_point= nearest_neighbors(distance_point, K) 
      y_pred_point    = voting(df_nearest_point, y_train) 
      y_pred.append(y_pred_point)

    return y_pred  


In [12]:
K=3
y_pred_one= KNN(normalized_x_train, y_train, normalized_x_test, K)
print(y_pred_one)

[2.0, 1.0, 0.0, 2.0, 0.0, 2.0, 0.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 1.0, 0.0]


In [13]:
knn=KNeighborsClassifier(K)
knn.fit(normalized_x_train, y_train)
y_pred_sklearn= knn.predict(normalized_x_test)
print(y_pred_sklearn)

[2. 1. 0. 2. 0. 2. 0. 1. 1. 1. 2. 1. 1. 1. 1. 0. 1. 2. 0. 0. 2. 1. 0. 0.
 2. 0. 0. 1. 1. 0.]


In [14]:
print(np.array_equal(y_pred_sklearn, y_pred_one))

True


In [15]:
print(f'The accuracy of our implementation is {accuracy_score(y_test, y_pred_one)}')
print(f'The accuracy of sklearn implementation is {accuracy_score(y_test, y_pred_sklearn)}')

The accuracy of our implementation is 0.9666666666666667
The accuracy of sklearn implementation is 0.9666666666666667


In [16]:
#Umnuh problemtoi hosluulan choosing cross validation hiih example
n_splits= 4 
kf= KFold(n_splits= n_splits) 

accuracy_k= [] 
k_values= list(range(1,30,2)) 
for k in k_values: 
  accuracy_fold= 0
  for normalized_x_train_fold_idx, normalized_x_valid_fold_idx in  kf.split(normalized_x_train): 
      normalized_x_train_fold= normalized_x_train[normalized_x_train_fold_idx] 
      y_train_fold= y_train[normalized_x_train_fold_idx]

      normalized_x_test_fold= normalized_x_train[normalized_x_valid_fold_idx]
      y_valid_fold= y_train[normalized_x_valid_fold_idx]
      y_pred_fold= KNN(normalized_x_train_fold, y_train_fold, normalized_x_test_fold, k)

      accuracy_fold+= accuracy_score (y_pred_fold, y_valid_fold)
  accuracy_fold= accuracy_fold/ n_splits 
  accuracy_k.append(accuracy_fold)

In [17]:
print(f'The accuracy for each K value was {list ( zip (accuracy_k, k_values))}') 
print(f'hamgiin sain ni {np.max(accuracy_k)}, niitseh K utga ni= {k_values[np.argmax(accuracy_k)]} ')

The accuracy for each K value was [(0.9666666666666668, 1), (0.9666666666666668, 3), (0.9666666666666668, 5), (0.9666666666666668, 7), (0.9583333333333333, 9), (0.9583333333333333, 11), (0.9583333333333334, 13), (0.95, 15), (0.9583333333333333, 17), (0.9583333333333333, 19), (0.9583333333333333, 21), (0.9583333333333333, 23), (0.9583333333333333, 25), (0.9583333333333333, 27), (0.9583333333333333, 29)]
hamgiin sain ni 0.9666666666666668, niitseh K utga ni= 1 
